In [1]:
import os

import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

from tfx.utils.dsl_utils import external_input
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen

In [2]:
# add project to the Pyithon path
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

os.environ["TFHUB_CACHE_DIR"] = '/Users/i530218/tmp/tfhub'

In [3]:
context = InteractiveContext()

In [4]:
base_dir = os.getcwd()
data_dir = "../data/"
examples = external_input(os.path.join(base_dir, data_dir))
example_gen = CsvExampleGen(input=examples)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/CsvExampleGen/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/CsvExampleGen/examples/1/eval/, split: eval, id: 3)]
        ))

In [5]:
""" Loading from GCP bucket
"""

# examples = external_input("gs://example_compliance_data/")
# example_gen = CsvExampleGen(input=examples)

# context.run(example_gen)

' Loading from GCP bucket\n'

In [6]:
""" Ingesting data from GCP BigQuery
"""

# !export GOOGLE_APPLICATION_CREDENTIALS=/Users/user/credentials.json
# or
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/user/credentials.json"

# from tfx.components.example_gen.big_query_example_gen.component import BigQueryExampleGen  

# query = """SELECT * FROM `complaint-pipeline.consumer_complaints_22Jan.complaints`"""
# example_gen = BigQueryExampleGen(query=query)

# context.run(example_gen)

' Ingesting data from GCP BigQuery\n'

In [7]:
""" Three-way data split
"""

# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# base_dir = os.getcwd()
# data_dir = "../data/"
# output = example_gen_pb2.Output( 
#              split_config=example_gen_pb2.SplitConfig(splits=[
#                  example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=6),
#                  example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=2),
#                  example_gen_pb2.SplitConfig.Split(name='test', hash_buckets=2)
#              ]))

# examples = external_input(os.path.join(base_dir, data_dir)) 
# example_gen = CsvExampleGen(input=examples, output_config=output)

# context.run(example_gen)

' Three-way data split\n'

In [8]:
for artifact in example_gen.outputs['examples'].get():
    print(artifact)

Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/CsvExampleGen/examples/1/train/, split: train, id: 2)
Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/CsvExampleGen/examples/1/eval/, split: eval, id: 3)


In [9]:
""" Input example
"""

# import os
# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
# context = InteractiveContext()

# base_dir = os.getcwd()
# data_dir = "../data/"

# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(name='train', pattern='train/*'),
#                 example_gen_pb2.Input.Split(name='eval', pattern='eval/*'),
#                 example_gen_pb2.Input.Split(name='test', pattern='test/*')
#             ])

# examples = external_input(os.path.join(base_dir, data_dir)) 
# example_gen = CsvExampleGen(input=examples, input_config=input)
# context.run(example_gen)

' Input example\n'

In [10]:
""" Span Loading Example
"""

# from tfx.utils.dsl_utils import external_input
# from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
# from  tfx.proto import example_gen_pb2

# base_dir = os.getcwd()
# data_dir = "../data3/"

# input = example_gen_pb2.Input(splits=[
#                 example_gen_pb2.Input.Split(pattern='export-{SPAN}/*')
#             ])

# examples = external_input(os.path.join(base_dir, data_dir)) 

# example_gen = CsvExampleGen(input=examples, input_config=input)
# context.run(example_gen)

' Span Loading Example\n'

In [11]:
from tfx.components.statistics_gen.component import StatisticsGen

statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [12]:
from tfx.components.schema_gen.component import SchemaGen

schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'company',BYTES,required,,-
'company_response',STRING,required,,'company_response'
'consumer_disputed',STRING,optional,single,'consumer_disputed'
'date_received',STRING,required,,'date_received'
'date_sent_to_company',STRING,required,,'date_sent_to_company'
'issue',STRING,optional,single,'issue'
'product',STRING,required,,'product'
'state',STRING,optional,single,'state'
'sub_issue',STRING,optional,single,'sub_issue'


,Values
Domain,
'company_response',"'Closed', 'Closed with explanation', 'Closed with monetary relief', 'Closed with non-monetary relief', 'In progress', 'Untimely response'"
'consumer_disputed',"'No', 'Yes'"
'date_received',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'date_sent_to_company',"'2015-01-01', '2015-01-02', '2015-01-03', '2015-01-04', '2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08', '2015-01-09', '2015-01-10', '2015-01-11', '2015-01-12', '2015-01-13', '2015-01-14', '2015-01-15', '2015-01-16', '2015-01-17', '2015-01-18', '2015-01-19', '2015-01-20', '2015-01-21', '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25', '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29', '2015-01-30', '2015-01-31', '2015-02-01', '2015-02-02', '2015-02-03', '2015-02-04', '2015-02-05', '2015-02-06', '2015-02-07', '2015-02-08', '2015-02-09', '2015-02-10', '2015-02-11', '2015-02-12', '2015-02-13', '2015-02-14', '2015-02-15', '2015-02-16', '2015-02-17', '2015-02-18', '2015-02-19', '2015-02-20', '2015-02-21', '2015-02-22', '2015-02-23', '2015-02-24', '2015-02-25', '2015-02-26', '2015-02-27', '2015-02-28', '2015-03-01', '2015-03-02', '2015-03-03', '2015-03-04', '2015-03-05', '2015-03-06', '2015-03-07', '2015-03-08', '2015-03-09', '2015-03-10', '2015-03-11', '2015-03-12', '2015-03-13', '2015-03-14', '2015-03-15', '2015-03-16', '2015-03-17', '2015-03-18', '2015-03-19'"
'issue',"'APR or interest rate', 'Account opening, closing, or management', 'Account terms and changes', 'Adding money', 'Advertising and marketing', 'Advertising, marketing or disclosures', 'Application processing delay', 'Application, originator, mortgage broker', 'Applied for loan/did not receive money', 'Arbitration', 'Balance transfer', 'Balance transfer fee', 'Bankruptcy', 'Billing disputes', 'Billing statement', 'Can\'t contact lender', 'Can\'t repay my loan', 'Can\'t stop charges to bank account', 'Cash advance fee', 'Charged bank acct wrong day or amt', 'Charged fees or interest I didn\'t expect', 'Closing/Cancelling account', 'Communication tactics', 'Cont\'d attempts collect debt not owed', 'Convenience checks', 'Credit card protection / Debt protection', 'Credit decision / Underwriting', 'Credit determination', 'Credit line increase/decrease', 'Credit monitoring or identity protection', 'Credit reporting company\'s investigation', 'Customer service / Customer relations', 'Customer service/Customer relations', 'Dealing with my lender or servicer', 'Delinquent account', 'Deposits and withdrawals', 'Disclosure verification of debt', 'Disclosures', 'Excessive fees', 'False statements or representation', 'Fees', 'Forbearance / Workout plans', 'Fraud or scam', 'Getting a loan', 'Identity theft / Fraud / Embezzlement', 'Improper contact or sharing of info', 'Improper use of my credit report', 'Incorrect exchange rate', 'Incorrect information on credit report', 'Incorrect/missing disclosures or info', 'Late fee', 'Lender repossessed or 

In [13]:
from tfx.components.example_validator.component import ExampleValidator

example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

,Anomaly short description,Anomaly long description
Feature name,,
'issue',Unexpected string values,"Examples contain values missing from the schema: Cash advance (<1%), Lender damaged or destroyed vehicle (<1%), Unexpected/Other fees (<1%)."


In [14]:
# %%skip_for_export
%%writefile transform.py

import tensorflow as tf 
import tensorflow_transform as tft


_ONE_HOT_FEATURE_KEYS = [
    "product", "company_response", "timely_response"
]

_ONE_HOT_FEATURE_DIMS = [11, 6, 2]

_OOV_SIZE = 10

_CATEGORICAL_FEATURE_KEYS = [
    "sub_product", "state", "zip_code", 
]

_MAX_CATEGORICAL_FEATURE_VALUES = [41, 59, 90000] 

_TEXT_FEATURE_KEYS = [
    "issue", "sub_issue", "company"
]

_LABEL_KEY = 'consumer_disputed'


def _transformed_name(key):
    return key + '_xf'


def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]


def _fill_in_missing(x, to_string=False, unk=""):
    """Replace missing values in a SparseTensor.

    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.

    Args:
      x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
        in the second dimension.

    Returns:
      A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string or to_string else 0

    if type(x) == tf.SparseTensor:
        x = tf.sparse.to_dense(
            tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]), default_value)
    return tf.squeeze(x, axis=1)


def preprocessing_text(text):
    # let's lower all input strings
    text = tf.strings.lower(text)
    
    # Before applying the word piece tokenization, let's remove unnecessary 
    # tokens. This regex_replace was suggested by Aurélien Geron in his 
    # TFX workshop at TensorFlow World 2019
    # https://github.com/tensorflow/workshops/blob/master/tfx_labs/Lab_10_Neural_Structured_Learning.ipynb
    text = tf.strings.regex_replace(text, r" '| '|^'|'$", " ")
    text = tf.strings.regex_replace(text, "[^a-z' ]", " ")
    return text


def convert_num_to_one_hot(label_tensor, num_labels=2):
    """
    Convert a label (0 or 1) into a one-hot vector
    Args:
        int: label_tensor (0 or 1)
    Returns
        label tensor
    """
    one_hot_tensor = tf.one_hot(label_tensor, num_labels)
    return tf.reshape(one_hot_tensor, [-1, num_labels])

def convert_label(x):
    keys_tensor = tf.constant(['No', '', 'Yes'])
    vals_tensor = tf.constant([0, 0, 1])
    table = tf.lookup.StaticHashTable(
        tf.lookup.KeyValueTensorInitializer(keys_tensor, vals_tensor), -1)
    converted_label = table.lookup(x)
    converted_label = tf.cast(converted_label, tf.float32)
    converted_label = tf.reshape(converted_label, [-1, 1])
    return converted_label


def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.

    Args:
      inputs: map from feature keys to raw not-yet-transformed features.

    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}

    for i, key in enumerate(_CATEGORICAL_FEATURE_KEYS):
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key], to_string=True),
            top_k=_MAX_CATEGORICAL_FEATURE_VALUES[i],
            num_oov_buckets=_OOV_SIZE)

    for i, key in enumerate(_ONE_HOT_FEATURE_KEYS):
        int_value = tft.compute_and_apply_vocabulary(
            _fill_in_missing(inputs[key], to_string=True),
            top_k=_ONE_HOT_FEATURE_DIMS[i] + 1)
        outputs[_transformed_name(key)] = convert_num_to_one_hot(
            int_value,
            num_labels=_ONE_HOT_FEATURE_DIMS[i] + 1
        )

    for key in _TEXT_FEATURE_KEYS:
        outputs[_transformed_name(key)] = preprocessing_text(
            _fill_in_missing(inputs[key], to_string=True)
        )

    # label conversion
    outputs[_transformed_name(_LABEL_KEY)] = convert_label(
        _fill_in_missing(inputs[_LABEL_KEY], to_string=True))

    return outputs



Overwriting transform.py


In [15]:
from tfx.components.transform.component import Transform
# from tfx.proto import example_gen_pb2, pusher_pb2, trainer_pb2

transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath("transform.py"))
context.run(transform)

Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Transform/transform_output/5/.temp_path/tftransform_tmp/74b65e5b691e49c49cfb59a772403677/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Transform/transform_output/5/.temp_path/tftransform_tmp/eca99902f01b4ebaacf77123b39f73d1/saved_model.pb
INFO:tensorflow:Saver not created because there 

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Transform/transform_output/5/, split: , id: 8)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Transform/transformed_examples/5/train/, split: train, id: 9)
            Artifact(type_name: ExamplesPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Transform/transformed_examples/5/eval/, split: eval, id: 10)]
        ))

In [16]:
# %%skip_for_export

import pprint
import tensorflow_data_validation as tfdv

tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = transform.outputs['transformed_examples'].get()[1].uri

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a TFRecordDataset to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
decoder = tfdv.TFExampleDecoder()

# Iterate over the first 3 records and decode them using a TFExampleDecoder
for tfrecord in dataset.take(1):
    serialized_example = tfrecord.numpy()
    example = decoder.decode(serialized_example)
    pp.pprint(example)

{'company_response_xf': array([1., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'company_xf': array([b'accounts receivable consultants inc '], dtype=object),
 'consumer_disputed_xf': array([0.], dtype=float32),
 'issue_xf': array([b"cont'd attempts collect debt not owed"], dtype=object),
 'product_xf': array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'state_xf': array([2]),
 'sub_issue_xf': array([b'debt is not mine'], dtype=object),
 'sub_product_xf': array([5]),
 'timely_response_xf': array([1., 0., 0.], dtype=float32),
 'zip_code_xf': array([1013])}


In [22]:
# %%skip_for_export
%%writefile trainer.py

import os  # pylint: disable=unused-import

import tensorflow as tf 
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from components.model import convert_model_to_estimator, get_model
from transform import _transformed_name, _LABEL_KEY

def _get_raw_feature_spec(schema):
    return schema_utils.schema_as_feature_spec(schema).feature_spec

def _gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.data.TFRecordDataset(
        filenames,
        compression_type='GZIP')

def _example_serving_receiver_fn(tf_transform_output, schema):
    """Build the serving in inputs.
    Args:
      tf_transform_output: A TFTransformOutput.
      schema: the schema of the input data.
    Returns:
      Tensorflow graph which parses examples, applying tf-transform to them.
    """

    received_features = {
        'product': tf.compat.v1.placeholder(shape=None, dtype=tf.string, name="product"),
        'company_response': tf.compat.v1.placeholder(shape=None, dtype=tf.string, name="company_response"),
        'timely_response': tf.compat.v1.placeholder(shape=None, dtype=tf.string, name="timely_response"),
        'sub_product': tf.compat.v1.sparse_placeholder(shape=None, dtype=tf.string, name="sub_product"),
        'state': tf.compat.v1.sparse_placeholder(shape=None, dtype=tf.string, name="state"),
        'zip_code': tf.compat.v1.placeholder(shape=None, dtype=tf.int64, name="zip_code"),
        'issue': tf.compat.v1.sparse_placeholder(shape=None, dtype=tf.string, name="issue"),
        'sub_issue': tf.compat.v1.sparse_placeholder(shape=None, dtype=tf.string, name="sub_issue"),
        'company': tf.compat.v1.placeholder(shape=None, dtype=tf.string, name="company")
    }
    transformed_features = tf_transform_output.transform_raw_features(
        raw_features=received_features, 
    )

    # remove the unnecessary label feature 
    transformed_features.pop(_transformed_name(_LABEL_KEY))

    return tf.estimator.export.ServingInputReceiver(
        transformed_features, received_features)

def _eval_input_receiver_fn(tf_transform_output, schema):
    """Build everything needed for the tf-model-analysis to run the model.

    Args:
      transform_output: directory in which the tf-transform model was written
        during the preprocessing step.
      schema: the schema of the input data.

    Returns:
      EvalInputReceiver function, which contains:
        - Tensorflow graph which parses raw untransformed features, applies the
          tf-transform preprocessing operators.
        - Set of raw, untransformed features.
        - Label against which predictions will be compared.
    """
    # Notice that the inputs are raw features, not transformed features here.
    raw_feature_spec = _get_raw_feature_spec(schema)
    
    serialized_tf_example = tf.compat.v1.placeholder(
          dtype=tf.string, shape=[None], name='input_example_tensor')

    # Add a parse_example operator to the tensorflow graph, which will parse
    # raw, untransformed, tf examples.
    features = tf.io.parse_example(serialized_tf_example, raw_feature_spec)

    # Now that we have our raw examples, process them through the tf-transform
    # function computed during the preprocessing step.
    transformed_features = tf_transform_output.transform_raw_features(
          features)

    # The key name MUST be 'examples'.
    receiver_tensors = {'examples': serialized_tf_example}

    # NOTE: Model is driven by transformed features (since training works on the
    # materialized output of TFT, but slicing will happen on raw features.
    features.update(transformed_features)
    
    print(features)

    # without this it adds all the raw columns too
    # seems weird though
    f = {}
    for k in features.keys():
        if "_xf" in k and k != "consumer_disputed_xf":
            f.update({k: features[k]})
        # features.pop(k)

    print(f)

    return tfma.export.EvalInputReceiver(
          features=f,
          receiver_tensors=receiver_tensors,
          labels=transformed_features[_transformed_name(_LABEL_KEY)])


def _input_fn(filenames, tf_transform_output, batch_size=200):
    """Generates features and labels for training or evaluation.

      Args:
        filenames: [str] list of CSV files to read data from.
        tf_transform_output: A TFTransformOutput.
        batch_size: int First dimension size of the Tensors returned by input_fn

      Returns:
        A (features, indices) tuple where features is a dictionary of
          Tensors, and indices is a single Tensor of label indices.
    """
    transformed_feature_spec = (
      tf_transform_output.transformed_feature_spec().copy())

    dataset = tf.data.experimental.make_batched_features_dataset(
        filenames, batch_size, transformed_feature_spec, reader=_gzip_reader_fn)

    transformed_features = tf.compat.v1.data.make_one_shot_iterator(dataset).get_next()
    # We pop the label because we do not want to use it as a feature while we're
    # training.
    return transformed_features, transformed_features.pop(
        _transformed_name(_LABEL_KEY))


# TFX will call this function
def trainer_fn(hparams, schema):
    """Build the estimator using the high level API.

    Args:
      hparams: Holds hyperparameters used to train the model as name/value pairs
      schema: Holds the schema of the training examples.

    Returns:
      A dict of the following:
        - estimator: The estimator that will be used for training and eval.
        - train_spec: Spec for training.
        - eval_spec: Spec for eval.
        - eval_input_receiver_fn: Input function for eval.
    """

    train_batch_size = 64
    eval_batch_size = 64
    
    tf_transform_output = tft.TFTransformOutput(hparams.transform_output)


    def train_input_fn(): return _input_fn(  # pylint: disable=g-long-lambda
        hparams.train_files,
        tf_transform_output,
        batch_size=train_batch_size)

    def eval_input_fn(): return _input_fn(  # pylint: disable=g-long-lambda
        hparams.eval_files,
        tf_transform_output,
        batch_size=eval_batch_size)

    train_spec = tf.estimator.TrainSpec(  # pylint: disable=g-long-lambda
        train_input_fn,
        max_steps=hparams.train_steps)

    def serving_receiver_fn(): return _example_serving_receiver_fn(  # pylint: disable=g-long-lambda
        tf_transform_output, schema)

    exporter = tf.estimator.FinalExporter(
        'complaint_classification', serving_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(
        eval_input_fn,
        steps=hparams.eval_steps,
        exporters=[exporter],
        name='complaint_classification-eval')

    run_config = tf.estimator.RunConfig(
        save_checkpoints_steps=999, keep_checkpoint_max=1)

    run_config = run_config.replace(model_dir=hparams.serving_model_dir)

    keras_model = get_model(show_summary=False)
    estimator = convert_model_to_estimator(keras_model, run_config)

    # Create an input receiver for TFMA processing
    def receiver_fn(): return _eval_input_receiver_fn(  # pylint: disable=g-long-lambda
        tf_transform_output, schema)
    
    return {
        'estimator': estimator,
        'train_spec': train_spec,
        'eval_spec': eval_spec,
        'eval_input_receiver_fn': receiver_fn
    }


Overwriting trainer.py


In [23]:
from tfx.components.trainer.component import Trainer
from tfx.proto import trainer_pb2

TRAINING_STEPS = 100
EVALUATION_STEPS = 100

trainer = Trainer(
    module_file=os.path.abspath("trainer.py"),
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_gen.outputs['schema'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=trainer_pb2.TrainArgs(num_steps=TRAINING_STEPS),
    eval_args=trainer_pb2.EvalArgs(num_steps=EVALUATION_STEPS))
context.run(trainer)

n/eager/function.py", line 1715, in _call_flat
    flat_outputs = forward_function.call(ctx, args_with_tangents)
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/eager/function.py", line 574, in call
    executor_type=executor_type)
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/functional_ops.py", line 866, in partitioned_call
    executor_type=executor_type)
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/gen_functional_ops.py", line 431, in partitioned_call
    executor_type=executor_type, name=name)
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/framework/op_def_library.py", line 705, in _apply_op_helper
    attr_value.func.CopyFrom(_MakeFunc(value, key))
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/framework/op_def_library.py", line 224, in _MakeFunc
    v.add_to

ExecutionResult(
    component_id: Trainer
    execution_id: 9
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Trainer/output/9/, split: , id: 11)]
        ))

In [25]:
from tfx.components.model_validator.component import ModelValidator

model_validator = ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'])
context.run(model_validator)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Trainer/output/9/eval_model_dir/1580932082/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.
INFO:tensorflow:ExampleCount post export metric: could not find any of the standard keys in predictions_dict (keys were: dict_keys(['dense_31']))
INFO:tensorflow:Using the first key from predictions_dict: dense_31


ExecutionResult(
    component_id: ModelValidator
    execution_id: 10
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/ModelValidator/blessing/10/, split: , id: 12)]
        ))

In [26]:
!mkdir -p /tmp/serving_model_dir

from tfx.components.pusher.component import Pusher
from tfx.proto import pusher_pb2

_serving_model_dir = "/tmp/serving_model_dir"

pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

Exception ignored in: <function CapturableResourceDeleter.__del__ at 0x12ca220e0>
Traceback (most recent call last):
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/training/tracking/tracking.py", line 191, in __del__
    self._destroy_resource()
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/eager/def_function.py", line 568, in __call__
    result = self._call(*args, **kwds)
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/eager/def_function.py", line 615, in _call
    self._initialize(args, kwds, add_initializers_to=initializers)
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/eager/def_function.py", line 497, in _initialize
    *args, **kwds))
  File "/Users/i530218/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/eager/function.py", line 2389, in _get_concrete_function_internal_garbage_collected
   

ExecutionResult(
    component_id: Pusher
    execution_id: 11
    outputs:
        model_push: Channel(
            type_name: ModelPushPath
            artifacts: [Artifact(type_name: ModelPushPath, uri: /var/folders/m7/4m3r4lb904q4pbgcb9d52rhm0000gn/T/tfx-interactive-2020-02-05T10_48_42.936054-werxxwx2/Pusher/model_push/11/, split: , id: 13)]
        ))

In [30]:
!cat /tmp/serving_model_dir/1580932079/assets/vocab_compute_and_apply_vocabulary_1_vocabulary

CA
FL
TX
NY
GA
NJ
PA
IL
OH
VA
MD
NC
MI
AZ
MA
WA
TN
CO
SC
MO
AL
OR
NV
WI
MN
IN
CT
LA
KY
OK
KS
DC
UT
MS
NM
DE
IA
AR
ME
NH
ID
NE
RI
HI
WV
SD
VT
PR
MT
WY
ND
AK
AP
AE
VI
GU
MH
PW
AS
